#  This is the program that does the training and classification based on the approach presented in the paper.


In [5]:
%%time
import numpy as np
import pandas as pd
import math
from math import sqrt

data=pd.read_csv("/home/aakash/Documents/UCRArchive_2018/Trace/Trace_TRAIN.tsv",sep="\t",header=None)
y=len(data.columns)
x=len(data[0])
sdata=set(data[0])
classes=len(sdata)
mina=[[math.inf for _ in range(y-1)] for _ in range(classes)]
maxa=[[-math.inf for _ in range(y-1)] for _ in range(classes)]
total=[[0 for _ in range(y-1)] for _ in range(classes)]
mean=[[0 for _ in range(y-1)] for _ in range(classes)]
data[0]=data[0]-1
for i in range(x):
    for j in range(1,y):
        if data[j][i] < mina[data[0][i]][j-1]:
            mina[data[0][i]][j-1] = data[j][i]
        if data[j][i] > maxa[data[0][i]][j-1]:
            maxa[data[0][i]][j-1]=data[j][i]
        total[data[0][i]][j-1]=total[data[0][i]][j-1]+data[j][i]
count={}
for k in range(x):
    if data[0][k] in count:
        count[data[0][k]]+=1
    else:
        count[data[0][k]]=1
for i in range(classes):
    for j in range(1, y):
        mean[i][j-1]=total[i][j-1]/count[i]
        
count

CPU times: user 3.35 s, sys: 12 ms, total: 3.37 s
Wall time: 3.42 s


{0: 26, 1: 21, 3: 31, 2: 22}

In [1]:
        
testdata=pd.read_csv("/home/aakash/Documents/UCRArchive_2018/Trace/Trace_TEST.tsv",sep="\t",header=None)
correct=0

for i in range(len(testdata)):
    
    u=[[0 for _ in range(y-1)] for _ in range(classes)]
    score=[0 for _ in range(classes)]
    mem=[0 for _ in range(classes)]
    for j in range(1, y):
        for k in range(classes):
            if testdata[j][i] < mean[k][j-1]:
                u[k][j-1]= 1-((mean[k][j-1]-testdata[j][i])/(mean[k][j-1]-mina[k][j-1]))
            else:
                u[k][j-1]= 1-((testdata[j][i]-mean[k][j-1])/(maxa[k][j-1]-mean[k][j-1]))
        
    for j in range(classes):
        for k in range(1, y):
            score[j]=score[j]+u[j][k-1]
    for j in range(classes):
        mem[j]=score[j]/(sum(score))
    predicted=mem.index(max(mem))
    if predicted==testdata[0][i]:
        correct=correct+1
    print("pred.{} real.{}".format(predicted,testdata[0][i]-1))
print(correct)
print(len(testdata))

NameError: name 'pd' is not defined

# This is program that does the classification based on the 1NN classifier using Euclidean distance and MaximumNorm distance.

In [8]:
len(testdata)


73

In [19]:
def Euclidean(row1, row2):
    distance = 0.0
    for i in range(1,len(row1.columns)):
        distance += (float(row1[i]) - float(row2[i]))**2
    return sqrt(distance)
def MaximumNorm(row1, row2):
    distance=0.0
    for i in range(1, len(row1)):
        distance=max(abs(row1[i]-row2[i]),l)
    return distance

In [20]:
def get_neighbor_euclidean(train, test_row):
    distances = list()
    for i in range(len(train)):
        dist = Euclidean(train[i:i+1], test_row)        
        distances.append((i, dist))
    distances.sort(key=lambda tup: tup[1])
    neighbor = distances[0][0]
    return neighbor
def get_neighbor_maxi(train, test_row):
    distances = list()
    for i in range(len(train)):
        dist = MaximumNorm(train[i:i+1], test_row)        
        distances.append((i, dist))
    distances.sort(key=lambda tup: tup[1])
    neighbor = distances[0][0]
    return neighbor

In [29]:
count=0
for i in range(len(testdata)):
    if data[0][get_neighbor_euclidean(data, testdata[i:i+1])]== testdata[0][i]-1:
        count+=1
print(count)

1100


In [27]:
count=0
for i in range(len(testdata)):
    if data[0][get_neighbor_maxi(data, testdata[i:i+1])]== testdata[0][i]-1:
        count+=1
print(count)

577


In [31]:
testdata


,0,1,2,3,4,5,6,7,8,9,...,75,76,77,78,79,80,81,82,83,84
0,2,-0.342971,-0.313041,-0.278998,-0.266379,-0.258249,-0.239380,-0.240004,-0.239376,-0.225352,...,-1.041712,-0.982737,-0.903577,-0.871315,-0.858517,-0.858942,-0.821831,-0.795969,-0.737660,-0.719023
1,1,-0.566380,-0.667902,-0.746947,-0.786947,-0.844087,-0.899324,-0.960512,-1.018368,-1.062176,...,-1.350740,-1.392645,1.102528,1.288712,1.184430,1.315856,1.360425,1.433947,1.730128,1.958287
2,2,-0.236042,-0.207425,-0.213435,-1.098996,0.321766,-0.578797,-0.837535,-1.001627,-0.773978,...,-2.035833,-2.242515,-2.432943,-0.664577,-0.719915,-0.541185,-0.580071,-0.899353,-1.215521,-1.325865
3,2,1.222500,1.029721,0.371278,-0.090633,-0.203244,-0.346409,-0.415976,-0.216492,-0.336480,...,-0.240086,-0.107627,0.009074,0.123528,0.246111,0.277475,0.323939,0.408914,0.379662,0.417792
4,1,-1.022556,-1.060102,-1.071255,-1.116264,-1.167049,-1.194680,-1.218567,-1.258373,-1.291031,...,-1.063070,-1.035963,0.436232,0.837282,1.408442,1.886734,1.319802,0.575913,0.896932,1.219999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1247,2,-0.893472,-0.832250,-0.876202,-1.006601,-0.724068,-0.559684,-0.363747,-0.139348,0.040146,...,-1.690919,-1.499763,-1.354011,-1.158364,-0.889937,-0.655344,-0.511019,-0.306678,-0.237454,-0.133746
1248,1,-1.035570,-1.066416,-1.079312,-0.887277,-1.075031,-1.114812,-1.156933,-1.083966,-1.182248,...,-1.158835,-1.171718,-0.183577,0.144976,0.397248,0.710431,0.986983,0.980335,1.043356,1.261406
1249,2,-0.128368,0.020477,0.043310,0.168652,0.185692,0.054735,0.036175,0.013334,0.009052,...,-0.273816,-0.166922,-0.010960,-0.013829,-0.190731,-0.197938,-0.164731,-0.114983,-0.153488,-0.182519
1250,2,-0.137098,-0.069321,-0.011841,0.034630,0.089795,0.185531,0.242654,0.268567,0.306743,...,-4.874610,0.238762,0.260473,0.263389,0.268567,0.251725,0.238762,0.243949,0.257944,0.267273
